In [ ]:
# Comparison with SQL
# Found here:
# https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

import pandas as pd
import numpy as np

In [ ]:
url = (
    "https://raw.github.com/pandas-dev"
    "/pandas/master/pandas/tests/io/data/csv/tips.csv"
)

In [ ]:
tips = pd.read_csv(url)

In [ ]:
tips

## SELECT
In SQL, selection is done using a comma-separated list of columns you’d like to select (or a * to select all columns):
<br>
~~~
SELECT total_bill, tip, smoker, time
FROM tips;
~~~

In [ ]:
tips[["total_bill", "tip", "smoker", "time"]]

In SQL, you can add a calculated column:
```
SELECT *, tip/total_bill as tip_rate
FROM tips;
```
With pandas, you can use the DataFrame.assign() method of a DataFrame to append a new column:

In [ ]:
tips.assign(tip_rate=tips["tip"] / tips["total_bill"])

## WHERE
Filtering in SQL is done via a WHERE clause.
```
SELECT *
FROM tips
WHERE time = 'Dinner';
```
DataFrames can be filtered in multiple ways; the most intuitive of which is using boolean indexing.

In [ ]:
tips[tips["total_bill"] > 10]

In [ ]:
is_dinner = tips["time"] == "Dinner"
is_dinner

In [ ]:
is_dinner.value_counts()

In [ ]:
tips[is_dinner]

Just like SQL’s OR and AND, multiple conditions can be passed to a DataFrame using | (OR) and & (AND).

Tips of more than $5 at Dinner meals:
```
SELECT *
FROM tips
WHERE time = 'Dinner' AND tip > 5.00;
```

In [ ]:
tips[(tips["time"] == "Dinner") & (tips["tip"] > 5.00)]

Tips by parties of at least 5 diners OR bill total was more than $45:
```
SELECT *
FROM tips
WHERE size >= 5 OR total_bill > 45;
```

In [ ]:
tips[(tips["size"] >= 5) | (tips["total_bill"] > 45)]

NULL checking is done using the notna() and isna() methods.

In [ ]:
frame = pd.DataFrame(
    {"col1": ["A", "B", np.NaN, "C", "D"], "col2": ["F", np.NaN, "G", "H", "I"]}
)

frame

Assume we have a table of the same structure as our DataFrame above. We can see only the records where col2 IS NULL with the following query:
```
SELECT *
FROM frame
WHERE col2 IS NULL;
```

In [ ]:
frame[frame["col2"].isna()]

Getting items where col1 IS NOT NULL can be done with notna().
```
SELECT *
FROM frame
WHERE col1 IS NOT NULL;
```

In [ ]:
frame[frame["col1"].notna()]

## GROUP BY
In pandas, SQL’s GROUP BY operations are performed using the similarly named groupby() method. groupby() typically refers to a process where we’d like to split a dataset into groups, apply some function (typically aggregation) , and then combine the groups together.

A common SQL operation would be getting the count of records in each group throughout a dataset. For instance, a query getting us the number of tips left by sex:
```
SELECT sex, count(*)
FROM tips
GROUP BY sex;
/*
Female     87
Male      157
*/
```
The pandas equivalent would be:

In [ ]:
tips.groupby("sex").size()

Notice that in the pandas code we used size() and not count(). This is because count() applies the function to each column, returning the number of NOT NULL records within each.

In [ ]:
tips.groupby("sex").count()

Alternatively, we could have applied the count() method to an individual column:

In [ ]:
tips.groupby("sex")["total_bill"].count()

Multiple functions can also be applied at once. For instance, say we’d like to see how tip amount differs by day of the week - <font color='red'>agg()</font> allows you to pass a dictionary to your grouped DataFrame, indicating which functions to apply to specific columns.
```
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;
/*
Fri   2.734737   19
Sat   2.993103   87
Sun   3.255132   76
Thu  2.771452   62
*/
```

In [ ]:
tips.groupby("day").agg({"tip": np.mean, "day": np.size})

Grouping by more than one column is done by passing a list of columns to the groupby() method.
```
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day;
/*
smoker day
No     Fri      4  2.812500
       Sat     45  3.102889
       Sun     57  3.167895
       Thu    45  2.673778
Yes    Fri     15  2.714000
       Sat     42  2.875476
       Sun     19  3.516842
       Thu    17  3.030000
*/
```

In [ ]:
tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})

## JOIN
JOINs can be performed with join() or merge(). By default, join() will join the DataFrames on their indices. Each method has parameters allowing you to specify the type of join to perform (LEFT, RIGHT, INNER, FULL) or the columns to join on (column names or indices).

In [ ]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

Assume we have two database tables of the same name and structure as our DataFrames.

Now let’s go over the various types of JOINs.

## INNER JOIN
```
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;
```

In [ ]:
# merge performs an INNER JOIN by default
pd.merge(df1, df2, on="key")

merge() also offers parameters for cases when you’d like to join one DataFrame’s column with another DataFrame’s index.

In [ ]:
indexed_df2 = df2.set_index("key")

pd.merge(df1, indexed_df2, left_on="key", right_index=True)

## LEFT OUTER JOIN
Show all records from <font color="red">df1</font>.
```
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [ ]:
pd.merge(df1, df2, on="key", how="left")

## RIGHT JOIN
Show all records from df2.
```
SELECT *
FROM df1
RIGHT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [ ]:
pd.merge(df1, df2, on="key", how="right")

## FULL JOIN
pandas also allows for FULL JOINs, which display both sides of the dataset, whether or not the joined columns find a match. As of writing, FULL JOINs are not supported in all RDBMS (MySQL).

Show all records from both tables.
```
SELECT *
FROM df1
FULL OUTER JOIN df2
  ON df1.key = df2.key;
```

In [ ]:
pd.merge(df1, df2, on="key", how="outer")

## Question: SQL query to find record with ID not in another table

Use LEFT JOIN
```
SELECT  a.*
FROM    table1 a
            LEFT JOIN table2 b
                on a.ID = b.ID
WHERE   b.id IS NULL
```

In [ ]:
a = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

b = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

In [ ]:
ab = pd.merge(a, b, on="key", how="left")
ab[ab['value_y'].isna()][['key']]

In [ ]:
A = pd.DataFrame({"key": ["A", "B", "C", "D"]})

B = pd.DataFrame({"key": ["B", "D", "D", "E"]})

In [ ]:
AB = pd.merge(A, B, on="key", how="left")
# ab[ab['value_y'].isna()][['key']]

In [ ]:
AB

In [ ]:
a = np.array([1, 2, 3, 2, 4, 1])
b = np.array([3, 4, 5, 6])
np.setdiff1d(a, b)

In [ ]:
a = A.key.unique()
b = B.key.unique()

In [ ]:
list(np.setdiff1d(a, b))